In [ ]:
# This is a notebook to do some basic data exploration and 
# run some bagging/boosted methods 
# on a sample data set to distinguish between Lyman Alpha Emitting 
# Galaxies and OII Emitting Galaxies.

#Author: Viviana Acquaviva

#License: BSD but really should be TBD - just be nice.

### Bagging and Boosting Methods

Random Forests and XGBoost!

In [ ]:
import pandas as pd
import numpy as np
import time
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, \
    GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
import warnings


#Just to make our life easier!
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')


    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Data import

Read the data in a data frame using pandas, take a look at them, check the size of the data set, rename columns to something easier to type.

In [ ]:
data = pd.read_csv('LAE_OII_CCA.txt', sep = '\t', comment = '#')

### Data exploration

Look at data properties divided by type to figure out some differences between LAEs and OIIs. Change settings to visualize all the columns in a data frame. Eliminate outliers.

In [ ]:
data.describe()

In [ ]:
data.groupby('type').describe()

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
data.groupby('type').describe(percentiles = [])

In [ ]:
plt.hist(data[data.type == 'LAE']['EW'], bins = 20, range = (0,500), alpha = 0.5, label = 'LAE');
plt.hist(data[data.type == 'OII']['EW'], bins = 20, range = (0,500), alpha = 0.5, label = 'OII');
plt.legend();

In [ ]:
seldata = data[(np.abs(stats.zscore(data.drop(['type'],axis=1))) < 3).all(axis=1)]

In [ ]:
seldata.shape, data.shape

In [ ]:
seldata.groupby('type').describe()

### Transform pandas data frame into a numpy array that can be fed to sklearn methods, create feature and target arrays, and standardize (not necessary for tree-based).

In [ ]:
le = LabelEncoder()

In [ ]:
newcol = le.fit_transform(seldata.type.values)

In [ ]:
newcol

In [ ]:
# Note: this assigns the positive (OII) and negative (LAE) class! 
# Need to check and if necessary, flip.

seldata.ix[:,'type'] = newcol

In [ ]:
seldata.head()

In [ ]:
X, y = seldata.drop('type',axis=1), seldata.type

In [ ]:
normalized_X = (X - X.mean())/X.std()

In [ ]:
normalized_X.describe()

## Ensemble Method 1: Random Forests

Let's start with a RF Classifier with standard parameters, use cross_val_score and cross_val_predict; visualize and plot the confusion matrix.

In [ ]:
#Verify precision and recall


### We can look at the ROC/AUC by using the "predict_proba" feature.

In [ ]:
#I need to call "fit" explicitly to do this, so I am defining a train/test split

#Inspired by https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,random_state=5)

probas = model.fit(Xtrain, ytrain).predict_proba(Xtest) #doing only on one fold

# Compute ROC curve and area under the curve

fpr, tpr, thresholds = metrics.roc_curve(ytest, probas[:, 1])

roc_auc = metrics.auc(fpr, tpr)

plt.plot(fpr, tpr, lw=2, label = 'AUC = %0.2f' % roc_auc)

plt.legend();


### Feature ranking

After the model has been fit, it will have the attribute "feature\_importances\_". We can look at the feature importance using the following code:

In [ ]:
model.fit(normalized_X,y) #note: this is not doing any train/test split, but fitting the entire data set 

model.feature_importances_

The code below plots the feature importances.

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(normalized_X.shape[1]):
    print("%d. feature: %s, %d (%f)" % (f + 1, normalized_X.columns[indices[f]], indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure(figsize=(16,6))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(normalized_X.shape[1]), indices)
plt.xlim([-1, normalized_X.shape[1]])
plt.show()

We need to take it with a grain of salt (especially when we have only a few) because information is often split if features are not independent.

### How to improve?

In [ ]:
model

#### Tree Parameters

The parameters associated to that are:

-  The minimum number of instances in a leaf node;

-  The minimum number of instances required in a split node;

- The maximum depth of tree.

-  The criterion chosen to decide whether a split is "worth it", expressed in terms of information gain;


#### Randomization Parameters

- The number of k < n features that are used in building trees.

- The re-sampling (boostrap) of the data set


#### Forest Parameters

The number of trees in the forest (n_estimators) can be adjusted, with the general understanding that more trees are better, but at some point performance will plateau, so one can find the trade-off between having more trees and lower runtime.

<b> TASKS (10 minutes) </b> 

-  Play with your favorite parameters to see if you can beat the benchmark performance above.

-  Now do the same thing, but using recall as your scoring method.

### Ensemble methods 2: Gradient Boosting Models

Gradient Boosting models are another ensemble method where weak learners (usually decision stumps) are combined together.

Unlike Random Forests, the model is built by <b> adding individual trees in a sequential fashion, </b>
but choosing which trees we add to the model in a way that minimizes the current loss function. The "Gradient" part refers to the fact that we try to move along the gradient of the objective function (by calculating its numerical derivative) as we add more trees.

The parameters depend on the particular implementation.

In the sklearn formulation, the parameters of each tree are essentially the same we saw above; additionally we have the "learning_rate" parameter, which dictates how much each tree contribute to the final estimator, and the "subsample" parameters, which allows one to use a < 1.0 fraction of samples.

I liked this blog post about parameter tuning for GBMs:

https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

#### We'll do the usual benchmarking with cross_val_score and check differences with RF:

<b> TASKS (10 minutes) </b>

-  Use the get_params() method to find out the names and signatures of different parameters, and their default values.

-  Play with your favorite parameters to see how much you can improve the benchmark performance above.

-  Compare the timings to Random Forests.

### Subtleties in parameter optimization (we'll see some next week)

-  Use cv_results to look at gradients along algorithms and build understanding;

-  Push the edges of your parameter grid search; 

-  Do nested cross validation to evaluate the generalization error in order to avoid leakage between the parameter optimization and the cross validation procedure. 

### My advice: Define your own evaluation metric 

This is an example of what we did for this paper (Leung, VA et al 2016), where x0 = 1 - precision and x1 = 1 - recall.

<img src="Formula_Leung.jpg" width="300"/>


#### How to do that in code?

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
def my_loss_func(y, ypred):
    return np.log(1+np.abs(y-ypred).max())

In [ ]:
model = RandomForestClassifier()

In [ ]:
cross_val_score(model, normalized_X, y, cv = cvmethod, \
               scoring = make_scorer(my_loss_func, greater_is_better=False))

### Summary

Which algorithm is best, and how you will optimize it, really depends on what you are trying to do.

Define your own evaluation metric and/or pick the one that works best for your problem and your data.

### Exercises :) 

### 1. Try out xgboost (vs sklearn's GBM)

Sometimes knowns as "regularized" GBM, more robust to overfitting.

Has more flexibility in defining weak learners, and objective function.

Reputation of being very fast.

From the same author as the one above:

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [ ]:
import xgboost

In [ ]:
model = xgboost.XGBClassifier()

In [ ]:
model.get_params()

### 2. Try out HistGradientBoostingClassifier 

(sklearn's newest implementation, inspired by Microsoft's LightGBM. Promises to be super fast
on large ( > 10,000) data sets, by turning numerical features into bins to limit the number of possible splits. Requires installing sklearn v 0.21.x... I didn't)



In [ ]:
#This is the starter code

# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier


### 3. Compare DT, RF, GBM on same data set (either this one, or variable stars).

Compare performance of RF, GBM to DT out of the box for the variable star data set. Are ensemble methods effective in reducing overfitting?

Note: if you'd like, you can use this handy function "checktraintest" that I wrote to evaluate the difference between train and test scores in a "cross-validate-y" fashion. The standard deviation helps determine if the difference is statistically significant.

In [ ]:
def checktraintest(X, y, model, ntrials =5, test_size = 0.2):
    
    """evaluates the difference between a classifier's train and test scores 
    in a "k-fold-y" fashion. Output means and std to help determine if 
    the difference is statistically significant. """

    scores_train = np.zeros(ntrials)
    scores_test = np.zeros(ntrials)

    for i in range(ntrials):
        X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=test_size, random_state=i)
        model.fit(X_train, y_train)
        pred_test = model.predict(X_test)
        pred_train = model.predict(X_train)

        scores_test[i] = (metrics.accuracy_score(y_test,pred_test))
        scores_train[i] =(metrics.accuracy_score(y_train,pred_train))

    print('Training scores '+str(scores_train.mean())+' +- '+str(scores_train.std()))
    print('Test scores '+str(scores_test.mean())+' +- '+str(scores_test.std()))